<a href="https://colab.research.google.com/github/Satyamaadi/python/blob/master/first_cnn_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [3]:
num_classes = 10
num_rows,num_cols,num_ch = 28,28,1
input_shape = (num_rows,num_cols,num_ch)

In [4]:
(train_data,train_label),(test_data,test_label) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
train_data = train_data/255.0
test_data = test_data/255.0

In [6]:
train_data = train_data.reshape(train_data.shape[0], *input_shape)
test_data = test_data.reshape(test_data.shape[0],*input_shape)

In [11]:
class SimpleConvololutionLayer(tf.keras.layers.Layer):
  def __init__(self,num_kernels=32,kernel_size=(3,3),strides=(1,1),use_bias = True):
    super().__init__()
    super.num_kernels = num_kernels
    super.kernel_size = kernel_size
    super.strides = strides
    supers.use_bias = use_bias

  def build(self,input_shape):
    num_input_channels = input_shape[-1]
    kernels_shape = (*self.kernel_size, num_input_channels, self.num_kernels)
    glorot_uni_initializer = tf.initializers.GlorotUniform()
    self.kernels = self.add_weight(name='kernels',
                                       shape=kernels_shape,
                                       initializer=glorot_uni_initializer,
                                       trainable=True)  # and we make the variable trainable.

    if self.use_bias:  # If bias should be added, we initialize its variable too:
        self.bias = self.add_weight(name='bias',
                                        shape=(self.num_kernels,),
                                        initializer='random_normal',  # e.g., using normal distribution.
                                        trainable=True)
            
  def call(self,inputs):
     z = tf.nn.conv2d(inputs, self.kernels, strides=[1, *self.strides, 1], padding='VALID')
     if self.use_bias:
       z = z + self.bias
        # Finally, we apply the activation function (e.g. ReLU):
       return tf.nn.relu(z)

  def get_config(self):
        """
        Helper function to define the layer and its parameters.
        :return:        Dictionary containing the layer's configuration
        """
        return {'num_kernels': self.num_kernels,
                'kernel_size': self.kernel_size,
                'strides': self.strides,
                'use_bias': self.use_bias}         




In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [13]:
class LeNet5(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet5, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu')
        self.conv2 = Conv2D(16, kernel_size=(5, 5), activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

In [15]:
model = LeNet5(num_classes)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
_ = model.predict(test_data[:10])

# But we can build the model manually otherwise, providing the batched
# input shape ourselves:
batched_input_shape = tf.TensorShape((None, *input_shape))
model.build(input_shape=batched_input_shape)

# Method to visualize the architecture of the network:
model.summary()

Model: "le_net5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  156       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  2416      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  48120     
_________________________________________________________________
dense_1 (Dense)              multiple                  10164     
_________________________________________________________________
dense_2 (Dense)              multiple                  850 

In [20]:
history = model.fit(train_data, train_label,
                    batch_size=32, epochs=80, validation_data=(test_data, test_label), 
                    verbose=2,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
)



Epoch 1/80
1875/1875 - 37s - loss: 0.5355 - accuracy: 0.8315 - val_loss: 0.1315 - val_accuracy: 0.9608
Epoch 2/80
1875/1875 - 37s - loss: 0.1306 - accuracy: 0.9603 - val_loss: 0.1008 - val_accuracy: 0.9680
Epoch 3/80
1875/1875 - 38s - loss: 0.0946 - accuracy: 0.9704 - val_loss: 0.0680 - val_accuracy: 0.9772
Epoch 4/80
1875/1875 - 37s - loss: 0.0766 - accuracy: 0.9758 - val_loss: 0.0618 - val_accuracy: 0.9799
Epoch 5/80


KeyboardInterrupt: ignored